## Chapter 4.1 Explorative data analysis codes


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
df = pd.read_csv('EDAschema.csv')
import plotly.express as px
cutoff_date = pd.Timestamp("2025-1-1")
df["CloseDateNew"] = pd.to_datetime(df["CloseDateNew"]) 
# Filter the rows
#filtered_df = df[(df["Created date"] > cutoff_date ) & (df["CloseDateNew"] > cutoff_date)]
data= df[df["CloseDateNew"] > cutoff_date]
data = data[data["IsClosed"] == 1]

## Exploratory Data Analysis (EDA) – Opportunity Dataset

In [ ]:
from IPython.display import display

# Prepare summary table
summary = pd.DataFrame({
    'Unique Values': data.nunique(dropna=True),
    'Missing Values': data.isnull().sum(),
    'Missing %': (data.isnull().sum() / len(data) * 100).round(2),
    'Data Type': data.dtypes.astype(str)
})

# Optional: sort by unique values
summary = summary.sort_values('Unique Values', ascending=False)

# Display as a DataFrame in the notebook
display(summary)

## 2. Data Quality & Missing Fields

Understanding data quality is crucial for reliable analysis and modeling. Below is a summary of missing values in the dataset.

In [ ]:
import pandas as pd
missing = data.isnull().sum()
missing_pct = (missing / len(data) * 100).round(2)
missing_summary = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
}).sort_values('Missing %', ascending=False)
# Show top 10 fields with most missing data
display(missing_summary.head(20))

In [ ]:
# Visualize missingness (top 10)
import matplotlib.pyplot as plt
top_missing = missing_summary.head(15)
plt.figure(figsize=(8,4))
plt.barh(top_missing.index[::-1], top_missing['Missing %'][::-1], color="#F43256")
plt.xlabel("Missing (%)")
plt.title("Top 10 Fields by Missing Data")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np

# Summary statistics for 'Value selling field proportion'
summary = data['Value selling field proportion'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
print(summary)

# Plot histogram
plt.figure(figsize=(7,4))
plt.hist(data['Value selling field proportion'].dropna(), bins=30, color="#A259EA", edgecolor='k')
plt.xlabel('Value Selling Field Proportion')
plt.ylabel('Number of Opportunities')
plt.title('Distribution of Value Selling Field Proportion')

# Format x-axis as percentage
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:.0f}%'.format(x*100)))
plt.xticks(np.linspace(0, 1, 6))  # 0%, 20%, 40%, 60%, 80%, 100%

plt.tight_layout()
plt.show()

## 3. Target Variable Distribution (IsWon)

Distribution of opportunity outcomes (Won/Lost).

In [ ]:
# Calculate counts and percentages
counts = data['IsWon'].value_counts().sort_index()
percent = (counts / counts.sum() * 100).round(1)
labels = ['Lost (0)', 'Won (1)']

# Bar plot with Matplotlib
fig, ax = plt.subplots(figsize=(5,4))
bars = ax.bar(labels, counts, color=['#F43256', '#FFB000'])
for bar, pct in zip(bars, percent):
    height = bar.get_height()
    ax.annotate(f'{pct}%', xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=12)
ax.set_ylabel('Number of Opportunities')
ax.set_title('Distribution of Opportunity Outcomes (IsWon)')
plt.tight_layout()
plt.show()

## 4. Time Range of Key Timestamps

Check the range and distribution of key timestamp fields.

In [ ]:
# Ensure datetime
data['Created date'] = pd.to_datetime(data['Created date'], errors='coerce')
data['CloseDateNew'] = pd.to_datetime(data['CloseDateNew'], errors='coerce')
print("Created date range:", data['Created date'].min(), "to", data['Created date'].max())
print("CloseDateNew range:", data['CloseDateNew'].min(), "to", data['CloseDateNew'].max())

In [ ]:
# Visualize opportunities created per month
created_month = data['Created date'].dt.to_period('M').value_counts().sort_index()
created_month = created_month.rename_axis('Month').reset_index(name='Opportunities Created')
fig, ax = plt.subplots(figsize=(16,8))
ax.bar(created_month['Month'].astype(str), created_month['Opportunities Created'], color="#29BEFD")
plt.xticks(rotation=45, ha='right')
ax.set_ylabel('Number of Opportunities')
ax.set_title('Opportunities Created per Month')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize opportunities created per month
created_month = data['CloseDateNew'].dt.to_period('M').value_counts().sort_index()
created_month = created_month.rename_axis('Month').reset_index(name='Opportunities Created')
fig, ax = plt.subplots(figsize=(16,8))
ax.bar(created_month['Month'].astype(str), created_month['Opportunities Created'], color="#29BEFD")
plt.xticks(rotation=45, ha='right')
ax.set_ylabel('Number of Opportunities')
ax.set_title('Opportunities Closed per Month')
plt.tight_layout()
plt.show()

In [ ]:
#pitäis katsoa mikä kenttien täyttöaste on jos aikarajus onkin created 2025-1 lähtien 

## 5. Time to Close Analysis

How long does it take to close opportunities?

In [ ]:
# Calculate time to close in days
data['time_to_close_days'] = (data['CloseDateNew'] - data['Created date']).dt.total_seconds() / (24*3600)
# Summary statistics
print(data['time_to_close_days'].describe(percentiles=[0.1,0.25,0.5,0.75,0.9,0.95,0.99]))
# Histogram
plt.figure(figsize=(7,4))
plt.hist(data['time_to_close_days'].dropna(), bins=50, color="#FFB000", edgecolor='k')
plt.xlabel('Time to Close (days)')
plt.ylabel('Number of Opportunities')
plt.title('Distribution of Time to Close')
plt.tight_layout()
plt.show()

## 6. Win Rate by Market Segment

Which segments have the highest and lowest win rates?

In [ ]:
# Calculate win rate by Market Segment (top 10 by volume)
segment_counts = data['MarketSegmentc'].value_counts().head(10).index
segment_df = data[data['MarketSegmentc'].isin(segment_counts)]
win_rate_by_seg = segment_df.groupby('MarketSegmentc')['IsWon'].mean().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8,4))
bars = ax.bar(win_rate_by_seg.index, win_rate_by_seg.values, color="#F46821")
ax.set_ylabel('Win Rate')
ax.set_xlabel('Market Segment')
ax.set_title('Win Rate by Market Segment (Top 10 by Volume)')
plt.ylim(0, 1)
plt.tight_layout()
plt.show()

## 7. Opportunity Value Analysis

Distribution of deal sizes.

In [ ]:
# Summary statistics
import matplotlib.ticker as mtick
pd.options.display.float_format = '{:,.2f}'.format
print(data['Oppo value'].describe(percentiles=[0.5,0.75,0.9,0.95,0.99]))
# Histogram (log scale if skewed)
plt.figure(figsize=(7,4))
plt.hist(data['Oppo value'].clip(lower=0), bins=50, color="#A259EA", edgecolor='k', log=False)
plt.xlabel('Opportunity Value (EUR)')
plt.ylabel('Number of Opportunities')
plt.gca().xaxis.set_major_formatter(
    mtick.FuncFormatter(lambda x, _: f'{int(x/1000):,}k')
)
plt.title('Distribution of Opportunity Value')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

recommended_fields = [
    'IsWon',  # Target
    'Oppo value',
    'MarketSegmentc',
    'BusinessType',
    'Industryc',
    'Subsegmentc',
    'Ownerrolec',
    'Paymentterm',
    'Legal entity',
    'Tenderflag',
    'BigDealAlertSent',
    'Value selling field proportion'
]

# Filter only columns present in your data and numeric/categorical encoded
fields_in_data = [f for f in recommended_fields if f in data.columns]

# For categorical fields, encode as category codes for correlation
df_corr = data[fields_in_data].copy()
for col in df_corr.select_dtypes(include='object').columns:
    df_corr[col] = df_corr[col].astype('category').cat.codes

# Compute correlation matrix for recommended fields
corr_recommended = df_corr.corr()

# Display as a heatmap
plt.figure(figsize=(10,7))
sns.heatmap(corr_recommended, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Correlation Matrix: Recommended ML Features')
plt.tight_layout()
plt.show()

In [ ]:
missing = data.isnull().sum()
missing_pct = (missing / len(data) * 100).round(2)
missing_summary = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
}).sort_values('Missing %', ascending=True)
# Show top 10 fields with most missing data
display(missing_summary)